# QR Decompositon

The QR decomposition is incredibly useful as it allow us to take advantage of the relationship between orthonormal vectors, or rather the orthonormal columns in a matrix. The QR decomposition is as follows...

$ A = QR $

Where $Q$ is an orthonomal matrix and $R$ is upper triangular.

The main restriction on performing a $QR$ decomposition is that our $A$ matrix must be a square matrix. While this restriction may be a drawback to some, the $QR$ decomposition still proves to be one of the most useful matrix decompositions (or factorizations, whatever you prefer)because of the orthonormal matrix's involvement in the decomposition. But, more importantly, the $QR$ decomposition is used frequently in applications of numerical methods and machine learning. Before discussing its applications and rather than talk about the utility of the $ QR $ decomposition, let's explore and better understand how to perform the decomposition with modern computation!

## Quick $QR$ with numpy

The following code shows us a quick $QR$ decomposition of a square matrix using the `numpy.linalg.qr` function which performs the decompostion without you needing to worry about how the backend works.

In [1]:
import numpy as np

A = np.array(np.random.randint (0, 10, (3, 3)), dtype=np.float64)
Q, R = np.linalg.qr(A)
print("Q:")
print(Q)
print("R:")
print(R)

qqt = np.dot(Q, Q.T)
qtq = np.dot(Q.T, Q)
print("Q Q**T:")
print(np.round(qqt))
print("Q**T Q:")
print(np.round(qtq))

Q:
[[-0.38348249  0.80957415 -0.44444444]
 [-0.69026849 -0.57096283 -0.44444444]
 [-0.61357199  0.13634933  0.77777778]]
R:
[[-13.03840481  -5.44545142 -12.80831531]
 [  0.           4.83187943   3.57064811]
 [  0.           0.          -0.44444444]]
Q Q**T:
[[ 1.  0. -0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Q**T Q:
[[ 1.  0. -0.]
 [ 0.  1. -0.]
 [-0. -0.  1.]]


As you can see our $Q$ matrix is orthonormal (by the $ I = Q Q^T = Q^T Q $ computation) and $R$ is upper triangular. While this performs the computation quickly and easily, it tells you nothing about the algorithms and mathematics used in performing the decomposition. For those that just want to quickly perform a $QR$ decomposition, here you go. Here's how you can do it in python. Otherwise let's proceed with talking through some of the background algorithms used in performing $QR$

Fair warning there is more than one way to perform the decomposition. So before going deeper into the weeds, let me just say you have been warned.

## QR by the Gram-Schmidt Process

The first method we can use to perform the $QR$ decomposition is by using the Gram-Schmidt process. The Gram-Schmidt process is a process by which we can take a random set of vectors (i.e. columns of a matrix) and produce a set of orthornormal vectors (i.e orthonormal columns), and thus produce an orthornomal matrix [see video for more info](https://youtu.be/SChiY-2MiWo). So just by using the Gram-Schmidt process we can already see how we are obtaining our orthonormal matrix ($Q$) in our $QR$ decomposition. To get the $R$ matrix for the decomposition we can just compute the inner products in the following matrix template for $R$.

$R = \begin{pmatrix}
  \langle q_1 | a_1 \rangle & \langle q_1 | a_2 \rangle & \cdots & \langle q_1 | a_n \rangle \\
  0 & \langle q_2 | a_2 \rangle & \cdots & \langle q_2 | a_n \rangle \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  0 & 0 & \cdots & \langle q_n | a_n \rangle 
 \end{pmatrix}$
 
 Recall that the Gram-Schmidt process uses projections to obtain an orhtonormal set of vectors (i.e. orhtonormal columns). Using the `gram_schmidt` function below we can obtain our $Q$ matrix and then just computue the necessary inner products using our $Q$ and $A$ matrices in the `qr_gs` function (stands for $QR$ by Gram-Schmidt).

In [31]:
print(f"Runtime: {end - start} ns")

def gram_schmidt(A):
    m, n  = A.shape
    Q = np.zeros((m, n))
    Q[:, 0] = A[:, 0] / np.linalg.norm(A[:, 0], 2)
    for i in range(1, n):
        Q[:, i] = A[:, i]
        for j in range(0, i):
            inner = np.dot(Q[:, j].T, Q[:, i])
            Q[:, i] = Q[:, i] - np.dot(inner, Q[:, j])
        Q[:, i] = Q[:, i] / np.linalg.norm(Q[:, i], 2)
    return Q

def qr_gs(A):
    m, n = A.shape
    Q = gram_schmidt(A)
    R = np.zeros((m, n))
    for i in range(0, n):
        R[i, i] = np.dot(Q[:, i], A[:, i])
        for j in range(0, i):
            R[j, i] = np.dot(Q[:, j], A[:, i])
    return Q, R
        
start = perf_counter_ns()
Q, R = qr_gs(A)
end = perf_counter_ns()
print("Q:")
print(Q)
print("R:")
print(R)
print(f"Runtime: {end - start} ns")

Runtime: 1320545 ns
Q:
[[ 0.38348249  0.80957415  0.44444444]
 [ 0.69026849 -0.57096283  0.44444444]
 [ 0.61357199  0.13634933 -0.77777778]]
R:
[[13.03840481  5.44545142 12.80831531]
 [ 0.          4.83187943  3.57064811]
 [ 0.          0.          0.44444444]]
Runtime: 552102 ns


Comparing these results to those before from the `numpy` function we can see that the results are the same except of the absense of some minus signs. This does not mean the that $QR$ decomposition failed by Gram-Schmidt. It just means that the `numpy` function is using a different backend method to compute $QR$. In short, computing $QR$ by Gram-Schmidt is subject to worse numerical error than other methods. I'd love to elaborate further but it would require going down a deep and dense path (which I certainly would like go down in the future), but for now I'll just have to leave the reasoning there for $QR$ by Gram-Schmidt. Why use it at all then? Because it is very easy to implement it.

## QR by Householder Transformations

The second way of performing $QR$ is by using Householder transformations. Without going too much into the details of Householder transformations, we can think of the process we'll use as a series of reflections when using Householder transformations. This is very similar to the projections used by Gram-Schmidt but will lead to more reliable results (i.e. less error). The process is a little bit tricky though.

First, lets explore the process we'll follow in a more general manner.

This process involves computing multiple $Q_n$ matrices one at a time (with each step/iteration) in the following manner.

$Q_n ... Q_2 Q_1 A = R$

Then taking advantage of orthonormal matrices we can matrix multiply each $Q_n^T$ together to get $Q$.

i.e.
$Q^T = Q_n ... Q_2 Q_1$ and $Q_1^T Q_2^T ... Q_n^T$ = $Q$

So then how do we find each $Q_n$?

First we'll compute $\|q_n \|_2$ of the first column of our matrix.
Next, we'll take that value and place it in the first position of a vector of zeros of $m$ dimension ($m$ is determined by $A_{m \times n}$. We'll then subtract $\vec{q_n}$ by this new vector we've established and call this $\vec{u}$.

$\vec{u}$ = $\vec{q_n}$ - $\begin{pmatrix}
  \|q_n \|_2 \\
  0 \\
  \vdots \\
  0  
 \end{pmatrix}$
 
 Then we'll normalize $\vec{u}$ and call this $\vec{v}$.
 
 $\vec{v} = \frac{\vec{u}} {\|u \|_2}$
 
 Then with this we can compute the first $Q_n$ matrix with the following.
 
 $Q_n = I - 2 \vec{v} \vec{v}^T$
 
 Then we restart this process to compute the next $Q_n$ matrix using a sub matrix of $Q_n ... Q_1 A$. Meaning, we will ignore the first $n$ rows and $n$ columns of $Q_n ... Q_1 A$ based on what $Q_n$ matrix we just computed. 
 Do note that computing each subsequent $Q_n$ will result in a matrix that is $n$ dimensions smaller, so we'll need to pad the matrix with the identity matrix.
 
 i.e.
 
 $\begin{pmatrix}
  1 & 0 & ... & 0 \\
  0 & \ddots & ...& 0\\
  \vdots & \vdots & (Q_n ... Q_1 A)' &\\
  0 & 0 & & 
 \end{pmatrix}$
 
 Where $(Q_n ... Q_1 A)'$ is the submatrix of the matrix in parenthesis.
 
 We'll continue computing these $Q_n$'s until we have $R$ (i.e. an upper triangular matrix).
 
This means that each $Q_n$ matrix is zeroing out an entire column below the diagonal of the matrix until we get to an upper triangular matrix when we compute $Q_n ... Q_2 Q_1 A = R$. So our algorithm will have $n-1$ $Q_n$ matrces computed for $A_{m \times n}$.
 
 In the below implemtation `qr_householder` you can see that the values we get for $Q$ and $R$ are the same except for some differing minus signs.
 To better understand what is going with the process consider re-running the below code with some `print()` statements in the loop to see the process unfold.

In [32]:
def qr_householder(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)
    for i in range(0, n - 1):
        alpha = np.linalg.norm(R[i:, i], 2)
        avec = np.zeros_like(R[i:, [i]])
        avec[0] = alpha
        u = R[i:, [i]] - avec
        v = u / np.linalg.norm(u, 2)
        Qn = np.identity(m - i) - (2 * np.dot(v, v.T))
        Qn = np.block([[np.eye(i), np.zeros((i, m - i))], 
                       [np.zeros((m - i, i)), Qn]])
        R = np.dot(Qn, R)
        Q = np.dot(Q, Qn.T)
    return Q, R
        
start = perf_counter_ns()
Q, R = qr_householder(A)
end = perf_counter_ns()
print("Q:")
print(Q)
print("R:")
print(np.round(R, 8))
print(f"Runtime: {end - start} ns")

Q:
[[ 0.38348249  0.80957415  0.44444444]
 [ 0.69026849 -0.57096283  0.44444444]
 [ 0.61357199  0.13634933 -0.77777778]]
R:
[[13.03840481  5.44545142 12.80831531]
 [ 0.          4.83187943  3.57064811]
 [ 0.          0.          0.44444444]]
Runtime: 908688 ns


So why choose to use Householder transformations to compute $QR$ rather than use the Gram-Schmidt process?
The short answer is better numerical error or better numerical stability. Again, the reasoning to explain this is dense and is better suited for a seperate discussion that involves the understanding of computing eigenvalues and signular values. Just know for now that although the Householder transformation method is more complicated and slower than Gram-Schmidt, its numerical error is superior compared to Gram-Schmidt.

## QR by Givens Rotations

The last and arguably best method to compute $QR$ is by use of Givens rotations. If you couldn't guess by the name, the Givens rotations method uses rotations to compute $QR$, rather than reflections as is done with the Householder transoformation method, or projections as is done with the Gram-Schmidt method. It was also developed by a numerical mathematician and computer scientist, Wallace Givens. So naturally you would expect it to be the best method of the 3 presented. 

The Givens Rotation method of computing $QR$ provides 2 key benefits over the Gram-Schmidt and Householder methods:

1. Great numerical stability (low error)
2. An algorithm that can be vectorized

The downside to the Givens rotation method of computing $QR$ is that it is complicated to understand and implement.

Let's explore the general process in a more general manner again.
Like $QR$ by Houseolder transformations, the general idea with Givens rotations is to compute a number of $G_n$ matrices in a similar manner to $Q_n$ matrices as used with Householder.

$G_n ... G_2 G_1 A = R$

Then,

$G_1^T G_2^T ... G_n^T$ = $Q$

This is very similar to the Householder method.

However, instead of each one of the $G_n$ matrices representing a reflection as is the case with $Q_n$ in the Householder method, these $G_n$ matrices represent rotations. Also, rather than each one of these $G_n$ matrices zeroing out an entire column below a diagonal (like $Q_n$'s with Householder does), they zero out a single value of a column below the diagonal until we obtain our $R$ upper triangular matrix. This means there is more work (more iterations) done to compute $QR$ with Givens rotations.

So then how do we go about performing the necessary computations? Better yet what values even goes into $G_n$ matrices? The short answer is $\sin$s and $\cos$s.

To do this we'll use basic trigonometry and the Pythagorean Theorem, working through our matrix one column at a time.

1. Gather the value along the diagonal of the matrix for the column you are on, and gather the value in your column that you need to zero out. These are to be considered your adjacent and opposite sides on a right triangle respectively.

Example: for the following matrix this would be (a and d), (a and g), and (e and h).
$\begin{pmatrix}
  a & b & c\\
  d & e & f\\
  g & h & i
 \end{pmatrix}$
 
 2. Compute the hypotenuse of this right triangle using $a^2 + b^2 = c^2$, where c is your hypotenuse.
 
 3. Now compute $\sin$ and $\cos$ for this pair of values from your matrix with $\sin{\theta} = \frac{-b} {c}$ and $\cos{\theta} = \frac{a} {c}$
 
 4. Place these into a rotation matrix $G_n$
 
 And this is actually where we will diverge slightly from the mathematical theory. Although we could construct an entire rotation matrix and compute things with matrix multiplication, there is a better way. That better way is elementwise computation. It will use less computer memory, and will allow us to only perform the computations that we need. Doing this though is not the most intuitive.
 
 5. Perform the following multiplications of the $i^{th}$ and $j^{th}$ rows by the respective $\sin$'s and $\cos$'s as shown in the below code to establish $R$
 
 6. Perform the following multiplications of the $i^{th}$ and $j^{th}$ columns by the respective $\sin$'s and $\cos$'s as shown in the below code to establish $Q$
 
 **Do note that I would have wanted to spell this out if a much better manner but after numerous edits, the code seemed to be a clearer explanation of the process compared to the mathematics and words I had written out**

In [25]:
def givensrotation(a, b):
    hypot = np.sqrt(a**2 + b**2)
    cos = a / hypot
    sin = -b / hypot
    return cos, sin


def qr_givens(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation(R[i, i], R[j, i])
            R[i], R[j] = (R[i] * cos) + (R[j] * (-sin)), (R[i] * sin) + (R[j] * cos)
            Q[:, i], Q[:, j] = (Q[:, i] * cos) + (Q[:, j] * (-sin)), (Q[:, i] * sin) + (Q[:, j] * cos)
    return Q, R

start = perf_counter_ns()
Q, R = qr_givens(A)
end = perf_counter_ns()
print("Q:")
print(Q)
print("R:")
print(np.round(R, 8))
print(f"Runtime: {end - start} ns")

qqt = np.dot(Q, Q.T)
qtq = np.dot(Q.T, Q)
print("Q Q**T:")
print(np.round(qqt))
print("Q**T Q:")
print(np.round(qtq))

Q:
[[ 0.38348249  0.80957415  0.44444444]
 [ 0.69026849 -0.57096283  0.44444444]
 [ 0.61357199  0.13634933 -0.77777778]]
R:
[[13.03840481  5.44545142 12.80831531]
 [ 0.          4.83187943  3.57064811]
 [-0.          0.          0.44444444]]
Runtime: 230599 ns
Q Q**T:
[[ 1. -0. -0.]
 [-0.  1. -0.]
 [-0. -0.  1.]]
Q**T Q:
[[ 1. -0. -0.]
 [-0.  1. -0.]
 [-0. -0.  1.]]


And as you can see we get the same results as before (again without a few of the minus signs), and when checking our $Q$ matrix it is in fact orthornormal.

Like with the Householder method example, I would suggest adding in a print statement to both the i and j loops above to print out the R matrix particularly to see how the acutal process works.

Let's also shine some particular light on the following lines from the loops in the above code block.

```
R[i], R[j] = (R[i] * cos) + (R[j] * (-sin)), (R[i] * sin) + (R[j] * cos)
Q[:, i], Q[:, j] = (Q[:, i] * cos) + (Q[:, j] * (-sin)), (Q[:, i] * sin) + (Q[:, j] * cos)
```

These lines of code should look a little be odd to you. After all they do not follow PEP8. These computations of the rows of $R$ and the columns of $Q$ must take place in the same line so that they both take place at the same time. This is because of the fact that we are taking what should be done with matrix multiplication and mimicking that through elementwise computations. The added benefit here is computation speed.

This is what makes the Givens rotation method of computing a $QR$ decomposition so powerful. For the same reasons we would choose a $QR$ decomposion over other decomposition methods, we also should choose to perform the $QR$ decomposition by the Givens Rotation method. In the future we'll see just how much more powerful this decomposition is, but for now all this is how we can perform the $QR$ decomposition using 3 different methods.

## References

QR Decompostion Wikipedia page:

<https://en.wikipedia.org/wiki/QR_decomposition>

LAPACK users manual

<https://netlib.org/lapack/lug/node39.html>

Anderson, Edward (4 December 2000). "Discontinuous Plane Rotations and the Symmetric Eigenvalue Problem". LAPACK Working Note.

<https://www.netlib.org/lapack/lawnspdf/lawn150.pdf>